## Hetionet 

In [1]:
from pykeen.datasets import Hetionet
dataset = Hetionet()
dataset

Hetionet(num_entities=45158, num_relations=24, create_inverse_triples=False)

In [24]:
train_hetionet = dataset.training.triples 
valid_hetionet = dataset.validation.triples
test_hetionet = dataset.testing.triples

Reconstructing all label-based triples. This is expensive and rarely needed.
Reconstructing all label-based triples. This is expensive and rarely needed.
Reconstructing all label-based triples. This is expensive and rarely needed.


In [46]:
import pandas as pd

train_df = pd.DataFrame(train_hetionet, columns=["head", "relation", "tail"])
valid_df = pd.DataFrame(valid_hetionet, columns=["head", "relation", "tail"])
test_df = pd.DataFrame(test_hetionet, columns=["head", "relation", "tail"])

hetionet_df = pd.concat([train_df, valid_df, test_df])
hetionet_df

,head,relation,tail
0,Anatomy::UBERON:0000002,AdG,Gene::10005
1,Anatomy::UBERON:0000002,AdG,Gene::114804
2,Anatomy::UBERON:0000002,AdG,Gene::118670
3,Anatomy::UBERON:0000002,AdG,Gene::128989
4,Anatomy::UBERON:0000002,AdG,Gene::132851
...,...,...,...
225015,Gene::2260,GpBP,Biological Process::GO:0002764
225016,Gene::2,Gr>G,Gene::55893
225017,Anatomy::UBERON:0001017,AeG,Gene::92595
225018,Gene::89797,GiG,Gene::134637


In [50]:
hetionet_df["tail"] = hetionet_df["tail"].str.replace('Compound::','')
hetionet_df["head"] = hetionet_df["head"].str.replace('Compound::','')
hetionet_df


,head,relation,tail
0,Anatomy::UBERON:0000002,AdG,Gene::10005
1,Anatomy::UBERON:0000002,AdG,Gene::114804
2,Anatomy::UBERON:0000002,AdG,Gene::118670
3,Anatomy::UBERON:0000002,AdG,Gene::128989
4,Anatomy::UBERON:0000002,AdG,Gene::132851
...,...,...,...
225015,Gene::2260,GpBP,Biological Process::GO:0002764
225016,Gene::2,Gr>G,Gene::55893
225017,Anatomy::UBERON:0001017,AeG,Gene::92595
225018,Gene::89797,GiG,Gene::134637


In [51]:
hetionet_df[hetionet_df["head"].str.match("DB*")]

,head,relation,tail
1126,DB00014,CbG,Gene::2798
1127,DB00014,CcSE,Side Effect::C0000737
1128,DB00014,CcSE,Side Effect::C0002170
1129,DB00014,CcSE,Side Effect::C0002418
1130,DB00014,CcSE,Side Effect::C0002453
...,...,...,...
224967,Disease::DOID:13223,DdG,Gene::3661
224983,DB01083,CcSE,Side Effect::C0042029
225003,Disease::DOID:9352,DaG,Gene::3952
225008,Disease::DOID:1324,DpS,Symptom::D003221


In [54]:
hetionet_df.to_csv("../data/triplets/hetionet.tsv", sep="\t")